## 1. 곡 DB load


In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('music.csv')

## 2. 가사 전처리

### 2.1. 1차 전처리 : 줄바꿈 생략 및 특수문자 제거

In [3]:
def remove_newlines_and_special_chars(paragraph):
    paragraph = paragraph.replace("\n", " ")
    
    chars_to_remove = "!@#$%^&*()_+-=[]{};':\"<>,.?/\\|"
    
    return paragraph.translate(str.maketrans("", "", chars_to_remove))

In [4]:
# 가사 전처리 후 '전처리가사'컬럼 생성
data['전처리가사'] = np.nan
for i in range(len(data['가사'])):
    data['전처리가사'][i] = (remove_newlines_and_special_chars(data['가사'][i]))

/tmp/ipykernel_21343/638557695.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['전처리가사'][i] = (remove_newlines_and_special_chars(data['가사'][i]))


In [ ]:
# Single list of sentences
sentences = [text for text in data['전처리가사']]

### 2.2. Pororo 라이브러리 이용, 가사 축약

### 2.3. 전처리후 축약된 가사의 임베딩

In [ ]:
#가사 임베딩추출
embeddings = model.encode(sentences, convert_to_tensor=True)

In [ ]:
### 2.4 전처리한 결과값과 기존 가사 전문 결과값 비교분석

## 3. 모델링 & 학습

In [23]:
!pip install datasets
!pip install -U sentence-transformers

In [5]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import losses

In [6]:
### 3.1. train, validation, test 분할

In [7]:
klue_sts_train = load_dataset("klue", "sts", split='train[:90%]')
klue_sts_valid = load_dataset("klue", "sts", split='train[-10%:]') # train의 10%를 validation set으로 사용
klue_sts_test = load_dataset("klue", "sts", split='validation')

Found cached dataset klue (/home/lab09/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)
Found cached dataset klue (/home/lab09/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)
Found cached dataset klue (/home/lab09/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


### 3.2. InputExample 클래스를 통해서 data를 학습할 수 있는 형태로 변환

In [8]:
from sentence_transformers.readers import InputExample

def make_sts_input_example(dataset):
    ''' 
    Transform to InputExample
    ''' 
    input_examples = []
    for i, data in enumerate(dataset):
        sentence1 = data['sentence1']
        sentence2 = data['sentence2']
        score = (data['labels']['label']) / 5.0  # normalize 0 to 5
        input_examples.append(InputExample(texts=[sentence1, sentence2], label=score))

    return input_examples
    
sts_train_examples = make_sts_input_example(klue_sts_train)
sts_valid_examples = make_sts_input_example(klue_sts_valid)
sts_test_examples = make_sts_input_example(klue_sts_test)

### 3.3. 파인 튜닝

#### 3.3.1. DataLoader 및 validation,test 검증기 생성

In [9]:
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# Train Dataloader
train_dataloader = DataLoader(
    sts_train_examples,
    shuffle=True,
    batch_size=32, # 32 (논문에서는 16)
)

# Evaluator by sts-validation
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    sts_valid_examples,
    name="sts-dev",
)

# Evaluator by sts-test
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    sts_test_examples,
    name="sts-test",
)

#### 3.3.2. 모델 설정을 위한 임베딩, 풀링

In [29]:
from sentence_transformers import SentenceTransformer, models

# Load Embedding Model
embedding_model = models.Transformer(
    model_name_or_path="klue/roberta-base", 
    max_seq_length=256,
    do_lower_case=True
)

# Only use Mean Pooling -> Pooling all token embedding vectors of sentence.
pooling_model = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

model = SentenceTransformer(modules=[embedding_model, pooling_model])

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for

### 3.4. 모델 학습 (sts data)

In [12]:
from sentence_transformers import losses
import math
# config
sts_num_epochs = 4
train_batch_size = 32
sts_model_save_path = "model"

# Use CosineSimilarityLoss
train_loss = losses.CosineSimilarityLoss(model=model)
# linear learning-rate warmup steps
warmup_steps = math.ceil(len(sts_train_examples) * sts_num_epochs / train_batch_size * 0.1) #10% of train data for warm-up
# Training
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=dev_evaluator,
    epochs=sts_num_epochs,
    evaluation_steps=int(len(train_dataloader)*0.1),
    warmup_steps=warmup_steps,
    output_path=sts_model_save_path,
    save_best_model=True
)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/329 [00:00<?, ?it/s]

Iteration:   0%|          | 0/329 [00:00<?, ?it/s]

Iteration:   0%|          | 0/329 [00:00<?, ?it/s]

Iteration:   0%|          | 0/329 [00:00<?, ?it/s]

### 3.5. 모델 저장

In [14]:
#모델 저장
model.save('model/train model')

In [12]:
# 파인 튜닝된, 모델 불러오기
model = SentenceTransformer('./sbert_model/train_model')

In [13]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': True}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

### 3.6. 모델 정확도 수치 확인

In [ ]:
test_evaluator(model, output_path=sts_model_save_path)

### 3.7. 임베딩값 피클에 저장

In [14]:
import pickle

In [16]:
#임베딩 불러오기
with open('./embeddings.pickle', 'rb') as f:
    embeddings = pickle.load(f)

## 4. 검색 프로세스

#### 4-1. 띄워쓰기에 제약을 받지않고 검색을 할수 있도록 해주는 로직

In [17]:
# # 띄워쓰기에 제약을 받지않고 검색을 할수 있도록 해주는 로직
# data['전처리가사2'] = data['전처리가사'].apply(lambda x:''.join(x.split())).str.lower()
# data['제목2'] = data['제목'].apply(lambda x:''.join(x.split())).str.lower()
# data['가수2'] = data['가수'].apply(lambda x:''.join(x.split())).str.lower()
# data['제목+가수'] = (data['제목'] + data['가수']).apply(lambda x:''.join(x.split())).str.lower()
# data['가수+제목'] = (data['가수'] + data['제목']).apply(lambda x:''.join(x.split())).str.lower()

#### 4-2 제목, 가수, 가사, 제목+가수, 가수+제목을 입력받아 검색할수 있도록 해주는 로직

In [18]:
# # 제목, 가수, 가사, 제목+가수, 가수+제목 검색이 들어왔을경우 포함된값을 뽑아준다.
# all_search = input('추천 받으실 곡을 입력하시오: ').lower()
# all_search = ''.join(all_search.split(' '))

# #검색한 결과값
# recommend_music = pd.concat([data[data['제목2'].str.contains(f'{all_search}')][['제목','가수','전처리가사']], data[data['가수2'].str.contains(f'{all_search}')][['제목','가수','전처리가사']], data[data['전처리가사2'].str.contains(f'{all_search}')][['제목','가수','전처리가사']],data[data['제목+가수'].str.contains(f'{all_search}')][['제목','가수','전처리가사']], data[data['가수+제목'].str.contains(f'{all_search}')][['제목','가수','전처리가사']]])
# recommend_music = recommend_music.drop_duplicates()

추천 받으실 곡을 입력하시오:  안녕


In [20]:
data

,제목,가수,발매일,장르,가사,이미지,전처리가사,전처리가사2,제목2,가수2,제목+가수,가수+제목
0,날 사랑했다면,박학기,1993.10.01,발라드,떠나버린 널 깨닫기엔\n하늘이 너무 맑아\n우리에게 필요한 건 시간일뿐\n\n영화속...,https://cdnimg.melon.co.kr/cm/album/images/005...,떠나버린 널 깨닫기엔 하늘이 너무 맑아 우리에게 필요한 건 시간일뿐 영화속의 사랑...,떠나버린널깨닫기엔하늘이너무맑아우리에게필요한건시간일뿐영화속의사랑얘기도힘든순간이있어우린...,날사랑했다면,박학기,날사랑했다면박학기,박학기날사랑했다면
1,그대 떠난 이밤에,옥슨 80,1981.09.25,록/메탈,바람부네 아픈 내맘에\n아름다운 추억 못잊어 부네\n그대 떠난 이밤\n그대 떠난 이...,https://cdnimg.melon.co.kr/cm/album/images/000...,바람부네 아픈 내맘에 아름다운 추억 못잊어 부네 그대 떠난 이밤 그대 떠난 이밤이 ...,바람부네아픈내맘에아름다운추억못잊어부네그대떠난이밤그대떠난이밤이깊어갈수록사랑의노래울려퍼...,그대떠난이밤에,옥슨80,그대떠난이밤에옥슨80,옥슨80그대떠난이밤에
2,나는 홀로 있어도,유심초,1980.11.01,성인가요/트로트,하얀 불빛 아래에\n침묵만이 흐르고\n낯설은 네 눈길에\n눈물만이 흐르네\n멀어져 ...,https://cdnimg.melon.co.kr/cm/album/images/003...,하얀 불빛 아래에 침묵만이 흐르고 낯설은 네 눈길에 눈물만이 흐르네 멀어져 간 발길...,하얀불빛아래에침묵만이흐르고낯설은네눈길에눈물만이흐르네멀어져간발길이다시올것만같애기다리는...,나는홀로있어도,유심초,나는홀로있어도유심초,유심초나는홀로있어도
3,Love Is (Dance Mega Mix Ver.),터보,2001.06.20,댄스,우린 6년전에 만났지\n\n널 사랑하게 됐어\n\n내 마음을 숨긴체\n\n널 따라 ...,https://cdnimg.melon.co.kr/cm/album/images/003...,우린 6년전에 만났지 널 사랑하게 됐어 내 마음을 숨긴체 널 따라 다녀었지 ...,우린6년전에만났지널사랑하게됐어내마음을숨긴체널따라다녀었지내친구는나를위해애썼고마침내내사...,loveis(dancemegamixver.),터보,loveis(dancemegamixver.)터보,터보loveis(dancemegamixver.)
4,사랑밖엔 난 몰라 (Original Ver.),심수봉,2009.08.13,성인가요/트로트,그대 내 곁에선 순간\n\n그 눈빛이 너무 좋아\n\n어제는 울었지만\n\n오늘은 ...,https://cdnimg.melon.co.kr/cm/album/images/006...,그대 내 곁에선 순간 그 눈빛이 너무 좋아 어제는 울었지만 오늘은 당신 땜에 ...,그대내곁에선순간그눈빛이너무좋아어제는울었지만오늘은당신땜에내일은행복할꺼야얼굴도아니멋도아...,사랑밖엔난몰라(originalver.),심수봉,사랑밖엔난몰라(originalver.)심수봉,심수봉사랑밖엔난몰라(originalver.)
...,...,...,...,...,...,...,...,...,...,...,...,...
4406,작은 기다림,쿨 (COOL),1995.10.01,댄스,널 이렇게 보내 줄 수\n밖에 없었어\n나 후회할 지도 모른 채\n끝없이 다가오는 ...,https://cdnimg.melon.co.kr/cm/album/images/000...,널 이렇게 보내 줄 수 밖에 없었어 나 후회할 지도 모른 채 끝없이 다가오는 너와의...,널이렇게보내줄수밖에없었어나후회할지도모른채끝없이다가오는너와의기억을난잊으려고차마눈을감았...,작은기다림,쿨(cool),작은기다림쿨(cool),쿨(cool)작은기다림
4407,WE LIKE 2 PARTY,BIGBANG (빅뱅),2015.06.01,랩/힙합,오늘도 친구들이 왔어\nMAN HOW YOU BEEN WHATS UP\nAYE 여기...,https://cdnimg.melon.co.kr/cm/album/images/023...,오늘도 친구들이 왔어 MAN HOW YOU BEEN WHATS UP AYE 여기 한...,오늘도친구들이왔어manhowyoubeenwhatsupaye여기한잔만줄래제일늦게마시는...,welike2party,bigbang(빅뱅),welike2partybigbang(빅뱅),bigbang(빅뱅)welike2party
4408,내 손을 잡아,아이유,2011.05.25,"발라드, 국내드라마",느낌이 오잖아\n\n떨리고 있잖아\n\n언제까지 눈치만 볼 거니\n\n네 맘을 말해...,https://cdnimg.melon.co.kr/cm/album/images/012...,느낌이 오잖아 떨리고 있잖아 언제까지 눈치만 볼 거니 네 맘을 말해봐 딴청 ...,느낌이오잖아떨리고있잖아언제까지눈치만볼거니네맘을말해봐딴청피우지말란말이야네맘가는그대로지...,내손을잡아,아이유,내손을잡아아이유,아이유내손을잡아
4409,Lie (Radio Ver.),박화요비,2000.06.14,R&B/Soul,말해줄래 나를 위해라면\n이러진 말아\n아직 그댈 놓아주기에는\n너무 난 두려워\n...,https://cdnimg.melon.co.kr/cm/album/images/000...,말해줄래 나를 위해라면 이러진 말아 아직 그댈 놓아주기에는 너무 난 두려워 이미 지...,말해줄래나를위해라면이러진말아아직그댈놓아주기에는너무난두려워이미지나간일이아니라면숨기지말...,lie(radiover.),박화요비,lie(radiover.)박화요비,박화요비lie(radiover.)


In [19]:
#검색된 곡 출력
recommend_music

,제목,가수,전처리가사
71,안녕,신해철,선물가게의 포장지 처럼 예쁘게 꾸민 미소만으로 모두 반할거라 생각해도 그건 단지 착...
539,뜨거운 안녕 (Feat. 성시경),싸이 (PSY),PSY collaboration TOY introducing 성발라 불타 올랐던 남...
561,아빠 안녕,현미,비가 나리네 비가 나리네 돌아선 이가슴에 그님은 어데 그어느곳에 이마음 벗사려 떨어...
578,안녕,박혜경,외로운 날들이여 모두 다 안녕 내 마음속에 눈물들도 이제는 안녕 오 어제의 너는 바...
581,안녕 나야,포맨 (4MEN),안녕 나야 잘 지내지 요즘 날씨 많이 춥지 요즘 감기 독하던데 감기 조심하고 잠자려...
...,...,...,...
4234,한번 더 이별,성시경,뒤돌아 보면 너의 생각을 떠올린 게 언제였더라 숨 가쁘게 사는 건 무디게 했어 끝나...
4257,SMILEY (Feat. BIBI),YENA (최예나),울지 마 울지 마 어린아이같이 웃는 게 웃는 게 이기는 거라고 You are so ...
4258,SoulMate (Feat. 아이유),지코 (ZICO),젊음의 한복판에서 두 남녀가 사는 작은 섬 네 기분이 여기 날씨고 새빨간 열매가 열...
4324,내사람,SG 워너비,내 가슴속에 사는 사람 내가 그토록 아끼는 사람 너무 소중해 마음껏 안아보지도 못했...


In [ ]:
####4.2 - 모델 성능 확인을 위한 검색 프로세스

In [127]:
# 검색된곡을 통하여 추천곡 10까지를 뽑아준다.
index4 = recommend_music.index[0]
pairs4 = []
for i in range(len(cosine_scores)):
    pairs4.append({'비교노래가수': data.가수[i], '비교노래제목':data.제목[i],'비교노래가사' : data.전처리가사[i], '가수': data.가수[index4] , '제목':data.제목[index4] ,'가사':data.전처리가사[index4],'발매일':data.발매일[i],'장르':data.장르[i],'이미지':data.이미지[i] ,'유사도' :cosine_scores[i][index4]})
pairs4 = sorted(pairs4, key=lambda x: x['유사도'], reverse=True)
data_re = pd.DataFrame(pairs4)
data_re[:11][['비교노래가수','비교노래제목','비교노래가사','발매일','장르','이미지']]

,비교노래가수,비교노래제목,비교노래가사,발매일,장르,이미지
0,조성모,잘가요... 내 사랑...,그죠 날 떠나실거죠 한번쯤 날 위해 솔직해줘요 가라시면 갈텐데 왜 애써 숨겼나요 아...,2001.09.13,발라드,https://cdnimg.melon.co.kr/cm/album/images/000...
1,이수영,그리고 사랑해,지금 돌아선 그대가 그대 맞는지 눈을 뜨고도 꿈을 꾸는 것 같아 떠나가라고 아프게 ...,2001.12.11,발라드,https://cdnimg.melon.co.kr/cm/album/images/000...
2,노을,떠나간다,나는 괜찮아 내가 널 잊어야 행복할 수 있다면 이미 떠나버린 니맘을 붙잡을 수는 없...,2012.04.19,발라드,https://cdnimg.melon.co.kr/cm/album/images/021...
3,더 크로스,"떠나가요, 떠나지마요",이제 더 이상 날 보지말아요 날 안아 줄 수없는거라면 이젠 그만 뒤돌아서 나를 떠나...,2005.06.15,발라드,https://cdnimg.melon.co.kr/cm/album/images/003...
4,이승환,그대가 그대를,참 많이 모질었었죠 나 당신께 나같지 않은 말땜에 놀랬겠죠 편할날이 없었죠 틀렸던 ...,2003.03.12,발라드,https://cdnimg.melon.co.kr/cm/album/images/000...
5,엠씨더맥스 (M.C the MAX),볼때기 사랑,길을 잃었죠 그대는 가네요 서둘러 이별이 오네요 많지 않은 바램과 많지 않은 약속에...,2007.01.04,록/메탈,https://cdnimg.melon.co.kr/cm/album/images/003...
6,박상민,해바라기 (그녀에게..),바람 소리에도 가슴은 글썽이나봐 그대일까 그대 보낸 미련일까 기다리는 꿈 속에서도 ...,2004.01.27,록/메탈,https://cdnimg.melon.co.kr/cm/album/images/000...
7,이수영,Never Again,말론 다 못하겠죠 이 내 마음을 처음 그대 보았을때 생각이 나요 어렵사리 키워온 나...,2001.02.02,발라드,https://cdnimg.melon.co.kr/cm/album/images/003...
8,씨야,여인의 향기,잘 지내 보여요 그대 얼굴은 날 잊고 잘 사는 듯 하죠 그런 그대가 얼마나 미운지 ...,2006.02.22,발라드,https://cdnimg.melon.co.kr/cm/album/images/003...
9,서태지와 아이들,난 알아요,난 알아요 이밤이 흐르고 흐르면 누군가가 나를 떠나 버려야 한다는 그 사실을 그 이...,1992.03.23,댄스,https://cdnimg.melon.co.kr/cm/album/images/000...
